# Automatic sleep stage classification based on EEG signals using discrete wavelet transform and ensemble methods

Project report for the course "Ensemble Methods", SS2017 at Universität Osnabrück, held by Johannes Leugering and Olivera Stojanovic

This report is handed in by Inga Ibs, Davide Valenti, Victoria Amo-Olea  and Felix Meyer zu Driehausen

## Introduction


The purpose of this project is to use ensemble methods to discriminate between different sleep stages within EEG sleep data. We experiment with Random Decision Forests and AdaBoost in order to compare the results and find the most appropriate approach with respect to our data.

Our general procedure is as follows: We first prepare and filter the data in order to reduce artifacts. Subsequently, we perform Wavelet decomposition on snippets of the data and further calculate the power of the resulting frequency bands.  The subsumption of these spectral energies for every channel of the EEG constitute our feature vector. Afterwards, we train and test two different ensemble methods; Random Decision Forest and AdaBoost. In order to optimize these results we randomized hyperparameter search and compare the results. To finish we propose and implement further methods to optimize classification.


## Sleep Data Description

The data was collected with the "[Traumschreiber](https://www.traumschreiber.uni-osnabrueck.de/)", a high-tech sleep mask developed for research purposes by Johannes Leugering and Kristoffer Appel at Universität Osnabrück.

The data used to train and test the classifier consists of six sets corresponding to different nights of sleep. Each data set comprises of information from seven Electroencephalogram (EEG) channels and one Electrocardiogram (ECG) channel, recorded for about seven hours of sleep.

The data is labeled by epochs of one second. Due to occasional bottlenecks in wireless transmisssion, there is no constant sampling rate. Anyhow, in most of the cases each second of data contains more than 200 datapoints. The labelling is according to the sleep stages introduced by the American Academy of Sleep Medicine (AASM), which differentiates between five main stages: 

1. Wakefulness: Active wakefulness with beta waves (+13 Hz) and relaxed wakefulness with mostly alpha waves (8-13 Hz).
2. Non-Rapid Eye Movement (NREM) 1: Dominated by Theta activity (4-7 Hz).
3. NREM-2: Characterized by Theta waves, sleep spindles and K-complexes.
4. NREM-3: Dominated by Delta wave (0.5-2 Hz) along with some sleep spindles.
5. Rapid Eye Movement (REM): Characterized by low-amplitude mixed-frequency brain waves. Theta, alpha and even beta activity can be observed.

For a concise treatment of sleep stage please refer to 

Silber, M. H., Ancoli-Israel, S., Bonnet, M. H., Chokroverty, S., Grigg-Damberger, M. M., Hirshkowitz, M., ... & Pressman, M. R. (2007). The visual scoring of sleep in adults. *J Clin Sleep Med*, 3(2), 121-131.



Now let us get startet with loading a few libraries. Besides some common libraries we require [PyWavelets](https://pywavelets.readthedocs.io/en/latest/) for performing Discrete Wavelet Decomposition as well as our own out-of-the-notebook feature extractor for the whole dataset. It it is included in this repository.

In [ ]:
import pywt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage    
import scipy.signal    
import os
from feature_extractor import *

In [ ]:
# read the data for subject a
data = pd.read_csv('../data/by_subject/a_data.csv')
labels = pd.read_csv('../data/by_subject/a_labels.csv')

### Preprocessing: Median filter

The data contains huge spikes that are probably product of interference in the Bluetooth signal. To eliminate peaks while altering the data as little as possible, we decided to implement a median filter, using the Scikit-learn median filter function, with a kernelsize of three. This filter runs through the signal datapoint by datapoint, replacing each datapoint by the median of neighboring entries.

In [ ]:
# Example Plot from data set 1, EEG channel 0
plt.figure(figsize = (15,5))
plt.plot(data['Ch0'])
plt.title('Channel 1 of Subject 1, whole night: Noisy peaks are clearly visible')
plt.ylabel('U/muV')
plt.xlabel('Time/Datapoints')
plt.show()

Therefore we simply apply the Median filter implemented to all channels

In [ ]:
def median_filter(data):
    processed_data = data.copy()
    all_channels_data = data[['Ch0','Ch1','Ch2','Ch3','Ch4','Ch5','Ch6','Ch7']]
    for ch in all_channels_data:
        processed_data[ch] = scipy.signal.medfilt(data[ch], kernel_size=3)
        
      
    return processed_data

pre_processed = median_filter(data)

The effect of the median filter on all channels of subject 1 can be confirmed in the plot below.

In [ ]:
# plot data of all channels before and after median filter

channels = ['Ch0','Ch1','Ch2','Ch3','Ch4','Ch5','Ch6','Ch7']
title = ['Channel 0','Channel 1','Channel 2','Channel 3','Channel 4','Channel 5','Channel 6','Channel 7']
color = ['red','blue','green','yellow','orange','black','purple', 'grey']

for i,channel in enumerate(channels):
    
    f, (ax1, ax2) = plt.subplots(2,figsize = (15,5), sharex=True, sharey=True)
    ax1.plot(data[channel], color = color[i])
    ax1.set_title(title[i]+' before median filter')
    f.subplots_adjust(hspace=.5)
    ax1.set_ylabel('U/muV')
    ax1.set_xlabel('Time/Datapoints')
    ax2.plot(pre_processed[channel], color = color[i])
    ax2.set_title(title[i]+' after median filter')
    ax2.set_ylabel('U/muV')
    ax2.set_xlabel('Time/Datapoints')
    plt.show()

We would like to work with chunks of the data of flexible size. For example, we would like to test on 1 second snippets of the data as well as the commonly used 30 second intervals. Therefore, we added a new column to the data and label dataframes, containing an identifier for each chunk. Now, the data and lables can be conveniently grouped by this column.

In [ ]:
# for snippt_length sec snipped grouping 
snippet_length = 3
data['TimestampGrouping'] = (data['Timestamp'] / snippet_length).astype(int)
labels['TimestampGrouping'] = (labels['Timestamp'] / snippet_length).astype(int)

labels_grouped = labels.groupby('TimestampGrouping')
labels_grouped_count = labels_grouped.count()
data_grouped = data.groupby('TimestampGrouping')

As the labels are for intervals of seconds, with varying snippet size this could lead to various labels with a snippet of data. We therefore discard snippets with multiple labels. We also discard snippets with too little data.

In [ ]:
# delete all rows for which there is too few data or that have several labels 
labels_grouped_count = labels_grouped_count[labels_grouped_count.Event == snippet_length]
labels_full_snippets = labels_grouped_count.index

Now that we have the data in a convenient format, we can start with feature extraction.

## Discrete Wavelet Transform

### Discrete Wavelet Transform Overview
The wavelets are waves of irregular form in shape and compactly supported, so they only act locally on the signal during convolution. These properties along with the main two operations of scaling and shifting, which produce a time-scale representation of the signal, make wavelets an ideal tool for analysing signals of non-stationary nature. Their irregular shape makes them suitable for analysing signals with discontinuities, and their compactly supported nature enables temporal localisation. Motivated by the adaptive time-frequency resolution properties of the Wavelet Transform and the corresponding fact that some stages in sleep recordings have a well defined time-frequency domain, we opted to use Discrete Wavelet Decomposition to obtain five sub-bands of the original signal and consequently performed feature extraction on them for the classification.

The Discrete Wavelet Decomposition algorithm we implemented relies firstly on a dyadic scaling of the wavelength and secondly on a discrete shifting across the original signal. The first operation serves as a half band filter that halves the highest frequency component of the original signal, providing lower computational time and less memory usage. This is in accordance to Nyquist’s sampling rate. It allows the usage of half of the previous sample points at each level of the decomposition for a proper reconstruction of the original signal. 

<img src="wavelet transform EEG ERD ERS event-related potentials time frequencya.jpg">

### Feature Extraction for sleep classification 

The code below performs the discrete wavelet transformation. It is adapted from [pwyt Examples](https://github.com/PyWavelets/pywt/blob/master/demo/dwt_signal_decomposition.py). While there is a shorter function call in the pywt API to achieve the same, we still leave the verbose version here in order to appreaciate the recursive application of the wavelet decomposition.

In [ ]:
mode = pywt.Modes.smooth

def signal_decomp(data):
    """Decompose and plot a signal S.
    S = An + Dn + Dn-1 + ... + D1
    """
    w = pywt.Wavelet('db4')
    a = data
    ca = []
    cd = []
    for i in range(5):
        (a, d) = pywt.dwt(a, w, mode)
        ca.append(a)
        cd.append(d)  
    return ca, cd

This function perform the calculation of the signal energy. It is a norm of the coefficients and therefore a measure for the presence of a specific freuency band within a signal. We will apply it seperately to the sets of coefficients which we obtained from the DWT for different frequency spectra. We took it from [here](https://stackoverflow.com/questions/37659422/energy-for-1-d-wavelet-in-python)

In [ ]:
def Energy(coeffs, k):
    return np.sqrt(np.sum(np.array(coeffs[-k]) ** 2)) / len(coeffs[-k])

### Signal Decomposition
The algorithm adopted can be better visualized as a tree of a low and high pass filter, which perform the decomposition of the signal into different frequency bands by applying successive filtering of the time domain signal. 
The original signal is successively decomposed into components of lower resolution, while the high frequency components are not analysed any further. This decomposition halves the time resolution since only half the number of samples now characterizes the entire signal. 
However it doubles the frequency resolution, since the frequency band of the signal now spans only half the previous. The maximum depth of decomposition is dependent on the input size of the data to be analysed, with 2N data samples enabling the breakdown of the signal into N discrete levels using the discrete wavelet transform. This procedure thus offers a good time resolution at high frequencies, and good frequency resolution at low frequencies. 

This matches well the resolution of each sub-band with certain sleep stages patterns, for example capturing at an higher time resolution the signal of the beta stage, which shows abrupt discontinuities.
We discarded the first level of the decomposition simply because those frequency bands are of no interest in the original signal. In conclusion the frequencies that are most prominent in the original signal will appear as high amplitudes in the corresponding region of the Discrete Wavelet Transform signal that includes those particular frequencies.

<img src="Untitled Diagram (3).png" width=80%>

The following performs the DWT on a chunk of data and plot reconstructed signals from different levels of approximate and detailed coefficients. The code is adapted from [pwyt Examples](https://github.com/PyWavelets/pywt/blob/master/demo/dwt_signal_decomposition.py)

In [ ]:
def plot_signal_decomp(data, w, title):
    ca, cd = signal_decomp(data)
        
    rec_a = []
    rec_d = []

    for i, coeff in enumerate(ca):
        coeff_list = [coeff, None] + [None] * i
        rec_a.append(pywt.waverec(coeff_list, w))

    for i, coeff in enumerate(cd):
        coeff_list = [None, coeff] + [None] * i
        rec_d.append(pywt.waverec(coeff_list, w))

    fig = plt.figure(figsize=(12,10))
    fig.subplots_adjust(hspace=.5)
    ax_main = fig.add_subplot(len(rec_a) + 1, 1, 1)
    ax_main.set_title(title, fontsize=20)
    ax_main.plot(data)
    ax_main.set_xlim(data.index[0], data.index[len(data) - 1])
    ax_main.set_title('The datasample')

    for i, y in enumerate(rec_a):
        ax = fig.add_subplot(len(rec_a) + 1, 2, 3 + i * 2)
        ax.plot(y, 'r')
        ax.set_xlim(0, len(y) - 1)
        ax.set_ylabel("A%d" % (i + 1))

    for i, y in enumerate(rec_d):
        ax = fig.add_subplot(len(rec_d) + 1, 2, 4 + i * 2)
        ax.plot(y, 'g')
        ax.set_xlim(0, len(y) - 1)
        ax.set_ylabel("D%d" % (i + 1))

In [ ]:
single_sec_data = data[-200:]
single_sec_ch = single_sec_data['Ch0']
plot_signal_decomp(single_sec_ch, 'db4', "DWT of a single second, single channel of data")
plt.show()

The plot nicely shows the decomposition of the signal by the discrete wavelet transform. On the left reconstructions from the approximate coefficients are displayed. Loosely put, A5+D5 combined are A4, A4+D4 = A3, A3+D3 = A2 ... 

## Feature extraction for one subject
In the following we prepare the feature and the target vector for one subject. We therefore apply the DWT to every snippet of the data and subsequently compute the power for all levels and channels. We concatenate these into a feature vector of 40 numbers (8 channels * 5 levels).

In [ ]:
# for every label, look up the corresponding data
features = []
for l in range(len(labels_full_snippets)):
    try:
        slice = data_grouped.get_group(labels_full_snippets[l])
    except KeyError:
        print(time)
        pass
    # for every channel
    power_all_channels = []
    # 1-7 EEG, 8th channel is ECG data
    for ch in range(8):
        snippet_ch = slice['Ch{}'.format(ch)]
        
        _, cd = signal_decomp(snippet_ch)
        # for every decomp. level
        power = []
        for l in range(5):
            power.append(Energy(cd, l))
            
        # collect power for all channels into one vector 
        power_all_channels.append(power) 
    # currently mean power of the frequency bands over all channels are the only features
    power_vec = np.asarray(power_all_channels).flatten()
    features.append(power_vec)
features = np.asarray(features)
 

Now it is time to set up the target vector corresponding to the features we just computed. We therefore look up the labels corresponding to the snippets and save them in a dataframe.

In [ ]:
targets = pd.DataFrame(index=labels_full_snippets)
events = []
for t in range(len(labels_full_snippets)):
    a = labels_grouped.get_group(labels_full_snippets[t])['Event']
    events.append(a.values[0])
targets['Event'] = events

Now we are all set to explore the classifiers!!!

## Classification within subjects and between subjects

In order to assess how good we can discriminate and predict the sleep stages from the calculated features, we have to distinguish two different objectives: firstly sleep stage classification within one subject and secondly sleep stage classficiation of separate subjects using generalisation over given subjects.

For the first problem we will explore the performance of our classifiers on two different data sets. One in which the train as well as the validation data consist of data from the same subject. With this set we will explore how good we can classify sleep stages from a subset of data of one night given our other date of the night for each subject separately. 
Another data set will be consisting of train and validation data from all the subjects and a separate test set separated from the whole data. There we want to see how much data we need in general in order to gain a good performance.

For the second problem we will split one subject as a test dataset and keep the rest as train and validation data. There we will also explore how much difference the length of the snippets (1s,3s,30s) on our performance.


### Classifier and general set up

We will test two different Ensemble method classification algorithms: Random Forest and AdaBoost using the sklearn implementation of these algorithms.

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

We will also use crossvalidation. In the case of within-subject-classification 10-fold crossvalidation for the hyperparameter search for the 30s snippet features and 10-fold crossvalidation for the hyperparameter search over all of the subjects data.
In the case of the classification of the stages for a separate subject we will use 4-fold LabelKFold crossvalidation which means that we always separate one subject as a validation set and use the rest of the data as a train set.


In [ ]:
from sklearn import ensemble
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import randint as sp_randint
from sklearn.cross_validation import LabelKFold
from sklearn.metrics import confusion_matrix
import itertools

### Load all the features
If no precomputed feature files are available, run the feature_extractor.py to get the feature files. In contrast to the feature extraction above, this includes all subjects.

In [ ]:
# get the feature and target table
features = pd.read_csv("../data/precomputed_features/features_3sec.csv")
targets = pd.read_csv("../data/precomputed_features/targets_3sec.csv")

### Feature statistics

In order to gain an insight into the variance we have over subjects we take a look at the mean and standard deviation of each subject. Below the mean and standard deviation for all of the 40 features are plotted, for the whole data and for each sleep phase. Each line corresponds to one subject.  We can see that the first and the last five features, corresponding to the first and last channel show the biggest differences in means and standard deviation, whereas the other channels are relatively similar.

In [ ]:
# merge the tables in order to derive the sleep phase later
merge = pd.merge(features, targets)

# group the data by each subject
grouped = merge.groupby('40')

# get the sleep phase identifier
events = merge['Event'].unique()

means = []
means_phase = np.zeros(shape=(6,5, 40))
stds_phase =  np.zeros(shape=(6,5,40))
stds = []

for i in range(6):
    g = grouped.get_group(i)
    
    del g['TimestampGrouping']
    del g['40']
    del g['subject_id']
    for j in range(events.shape[0]):
        p = g[g['Event']==events[j]]
        del p['Event']
        means_phase[i,j, :] = p.describe().loc['mean']
        stds_phase[i,j, :] = p.describe().loc['std']
        
    del g['Event']
    means.append(g.describe().loc['mean'])
    stds.append(g.describe().loc['std'])
        

# plot for all the features
plt.figure(figsize=(20,10))
ax1 = plt.subplot(121)
ax1.set_title('Means of the features for each subject', fontsize = 25)
ax1.plot(np.transpose(np.asarray(means)))

ax2 = plt.subplot(122)
ax2. set_title('Stds of the features for each subject', fontsize = 25)
ax2.plot(np.transpose(np.asarray(stds)))


# mean plot
fig=plt.figure(figsize=(20,20),facecolor='w', edgecolor='k')
fig.suptitle('Means for each sleep phase', fontsize = 25)
for i in range(5):
    temp = 320+i+1
    ax=plt.subplot(temp)
    ax.set_title(events[i], fontsize = 25)
    ax.plot(np.transpose(means_phase[:,i,:]))
plt.show()

# std plot
fig=plt.figure(figsize=(20,20),facecolor='w', edgecolor='k')
fig.suptitle('Stds for each sleep phase', fontsize = 25)
for i in range(5):
    temp = 320+i+1
    ax=plt.subplot(temp)
    ax.set_title(events[i], fontsize = 25)
    ax.plot(np.transpose(stds_phase[:,i,:]))
plt.show()


### Throwing out first and last channel

Given the statistics and assessing our prediction accuracy with the full dataset and with a dataset consisting only of channels 2-7 we came to the conclusion that dropping channel 1 and 8 can increase our performance accuracy. However we will drop these channels only in the prediction tasks in which we want to generalise over multiple subjects. Below we implement a function for dropping the channels and loading the features.

In [ ]:
def load_features(snippets = 3, drop = True):
    features = pd.read_csv("../data/precomputed_features/features_"+str(snippets)+"sec.csv")
    targets = pd.read_csv("../data/precomputed_features/targets_"+str(snippets)+"sec.csv")
    if drop == True:
        # throwing out the first and last channel due to high variance
        features = features.drop(features.columns[[1,2,3,4,5,36,37,38,39,40]], axis=1)
    return features, targets

### Reporting our results

For each classifier we will predict the accuracy of the prediction and plot the confusion matrices in order to assess how good the predictions are. 
For the hyperparameter search we will report the parameters of the three best models. The following cell contains code to plot confusion matrices nicley. It is from [here](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html)


In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=18)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=18)
    plt.yticks(tick_marks, classes, fontsize=18)

    if normalize:
        float_formatter = lambda x: "%.2f" % x
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, float_formatter(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black", fontsize=18)
        else:
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black", fontsize=18)

    plt.tight_layout()
    plt.ylabel('True label', fontsize=18)
    plt.xlabel('Predicted label', fontsize=18)

In [ ]:
# Utility function to report best scores for Random Search
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

### Hyperparameter search

We do a randomized hyperparameter search in order to approximate of which parameters our best model will consist. 

For the random forest we have as parameter set to test the number of trees in the forest (n_estimators) between 1 and 100 trees, the maximum depth of the trees (max_depth) being either 3 or no specified depth, as well as the maximal number of features considered for a split (max_features) which can be a number of features between 1 and 40. We also test whether bootstrapping changes the performance as well as the criterion with which the classifier tests for the best split, being either gini or entropy.

For the AdaBoost Classifier we test the number of trees in the set, the AdaBoost algorithm being either SAMME or SAMME.R and the kind of estimators (base_estimator) with which we test decision tree stumps and decision trees with maximum depth of 2.


In [ ]:
def do_random_search(X_train, y_train, classifier = 'rf', cv = 10):

    # run randomized search
    n_iter_search = 10
    
    if classifier == 'rf':
        ## Random Forst
        clf = ensemble.RandomForestClassifier(n_estimators = 10, criterion='entropy', class_weight='balanced', n_jobs = -1)

        # specify parameters and distributions to sample from
        param_dist = {"n_estimators":sp_randint(1, 100),
                  "max_depth": [3, None],
                  "max_features": [0.1,0.2,0.5,0.75,1.0, "auto"],
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    else:
        
        clf = ensemble.AdaBoostClassifier()
        # specify parameters and distributions to sample from
        param_dist = {"n_estimators":sp_randint(50, 250),
                      "algorithm": ["SAMME", "SAMME.R"],
                      "base_estimator": [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3)]
                     }

    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, cv = cv)

    random_search.fit(X_train, y_train['Event'])
    report(random_search.cv_results_)

### Classification performance within each subject
As a first experiment we evaluate our classifiers performance within each subject. We perform hyperparameter search on one subject by optimizing the accuracy of 10-fold crossvalidated predictions. We will use the hyperparameters obtained by this procedure to then train classifiers by 10-fold crossvalidations for all other subjects. 

In [ ]:
features, targets = load_features(snippets = 3, drop = False)

# group the data by each subject
grouped = features.groupby('40')
targets_grouped = targets.groupby('subject_id')

In [ ]:
features_a = grouped.get_group(0)
targets_a = targets_grouped.get_group(0)
# delete the column containing subject identifiers
del features_a['40']

In [ ]:
# Hyperparameter Search for one subject in order to approximate the best model
print("The best three models for the random forest are: \n")
do_random_search(features_a,targets_a, classifier = 'rf')

print("The best three models for the AdaBoost are: \n")
do_random_search(features_a,targets_a, classifier = 'ada')

We now use the parameters from the hyperparameter search in order to train models obtain prediction result for every subject:

In [ ]:
# run through all the subjects and calculate prediction accuracy with 5-fold crossvalidation
rf_predicted = []
rf_acc = []

ada_predicted = []
ada_acc = []

# insert parameters that worked best - see above
rf_clf = ensemble.RandomForestClassifier(max_depth = None, n_estimators = 82, criterion='entropy', class_weight='balanced', max_features=0.2, n_jobs = -1)
ada_clf = ensemble.AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=3),n_estimators=249, algorithm ='SAMME')

for i in range(6):
    g = grouped.get_group(i)
    del g ['40']
    t = targets_grouped.get_group(i)
    
    rf_predicted.append(cross_val_predict(rf_clf, g, t['Event'], cv=10))
    rf_acc.append( metrics.accuracy_score(t['Event'], rf_predicted[i]))
    
    ada_predicted.append(cross_val_predict(ada_clf, g, t['Event'], cv=10))
    ada_acc.append( metrics.accuracy_score(t['Event'], ada_predicted[i]))

print("The accuracy for each subject classification with the random forest are:")
print(rf_acc)
print("The mean accuracy is: {} \n".format(np.mean(rf_acc)))


print("The accuracy for each subject classification with the AdaBoost are:")
print(ada_acc)
print("The mean accuracy is: {}".format(np.mean(ada_acc)))

### Discussion of seperate subject model predictions
It certainly becomes clear that hyperparameter search yields to higher prediction accuracies. As we have done hyperparameter search only for one subject here, this subject get 90+% of accuracy while the others range between roughly 70% and 90%. This means that we are able to predict sleep stages with an accuracy of 90+%, if we train and test our classifier on non-overlapping random subsets of the same recording session (i.e a single night). This also includes hyper-parameter search. It is interesting to note, that we achieve these accuracies already on 1sec snippets of the data instead of the 30sec which are common practice in sleep research. This fact also made us wonder whether we are indirectly overfitting here. As we draw randomly from the whole night, it is very likely that our validation set comprises of examples which are in-between training examples with  respect to time. Neighbouring 1sec snippets of the data might be very similar, leading to the opporunity to overfit the training data and still perform very well on the validation data. We therefore will attempt inter-subject predictions, i.e generalisation over different recording sessions (as well as subjects) futher below in this report. 

### Smoothing Bayesian
Sleep stages change on a much slower timescale than we are attempting to do predictions. We can make use of this observation by appling a filter over our predictions that incorporates previous predictions in order to form the following. In this cell we implemented a proof of concept version of this filter. It performs a pointwise multiplication between the probability distribution over classes for the previous timestep (the prior) and the distribution over classes for the current timestep (likelihood). It further normalizes this product as to obtain a proper (posterior) distribution. We call it bayesian because it has a bayesian feel to us, but we did not formally prove anything that happens within this filter to be bayesian. We further weight current predictions higher (80%) than the described bayesian update (20%). The final prediction is the class with the max probability after this weighted sum. 

In [ ]:
def smooth(probabilities) :
    res = np.zeros(probabilities.shape)
    res[0,:] = np.squeeze(0.2 * np.ones((5,1)))
    for timep in range(probabilities.shape[0]):
        # from the second timepoint onwards
        if(timep > 0):
            prod = res[timep-1,:] * probabilities[timep,:]
            norm = np.sum(prod)
            res[timep,:] = ((prod / norm + 0.00002)*0.8 + probabilities[timep,:]*0.2)
            
    classes = np.argmax(res,axis=1)
    return classes

How we visualize...

In [ ]:
# visualize false predictions
def vis_clfs(targets, predicted, mode='string'):
    print(predicted.shape)
    # color coding grayscale
    if(mode=='string'):
        color = {'stage_q_N34' : 0, 'stage_q_N23': 50, 'stage_q_N12': 100, 'stage_q_REM1': 150, 'stage_q_Wake0': 200}
    elif(mode == 'int'):
        color = [100,50,0,150,200]
    label_text = ['N34', 'N23', 'N12', 'REM', 'Wake']
    false_pred = np.where(predicted != targets)
    timepoints = range(0,len(predicted))

    rows = np.ceil((len(predicted) / 500)).astype(int) * 10
    cols = 500
    image = np.ones((rows,cols), dtype=np.int16) * 255

    for timepoint in timepoints:
        x = timepoint % 500 
        y = int(timepoint / 500) * 10
        if(np.any(false_pred[0]==timepoint)):
            image[y:y+10,x] = 255
        else:
            image[y:y+10,x] = color[predicted[timepoint]]

    
    import matplotlib.patches as mpatches
    plt.figure(frameon=False, figsize=(16,16))  
    plt.title('Classification Results', fontsize=18)
    plt.axis('off')   
    im = plt.imshow(image,cmap=plt.cm.bone, vmin = 0, vmax = 255)
    # get the colors of the values, according to the 
    # colormap used by imshow
    values = [0,50,100,150,200]
    colors = [im.cmap(im.norm(value)) for value in values]
    # create a patch (proxy artist) for every color 
    patches = [ mpatches.Patch(color=colors[i], label="{l}".format(l=label_text[i]) ) for i in range(len(values)) ]
    # put those patched as legend-handles into the legend
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., fontsize=18 )
    plt.show()

Here we train the classifier only on 5% of of the data from subject 1 and further apply our filter to reach a 95+ accuracy.

In [ ]:
# do a train- test split on subject a

X_train, X_test, y_train, y_test = train_test_split(features_a, targets_a, test_size=0.95, random_state=42)
# sort the test set by timestamp for proper display
X_test = X_test.sort_values(by='TimestampGrouping')
y_test = y_test.sort_values(by='TimestampGrouping')

# fit the classifier on the training data
rf_clf.fit(X_train,y_train['Event'])

# calculate predictions and improved predictions
predictions = rf_clf.predict(X_test)
probabilities = rf_clf.predict_proba(X_test)
improved_predict = smooth(probabilities)

# change target labels for plot
number_targets = y_test['Event'].apply(lambda v: str(v).replace('stage_q_N12','0').replace('stage_q_N23','1').replace('stage_q_N34','2').replace('stage_q_REM1','3').replace('stage_q_Wake0','4'))
number_targets= number_targets.apply(int)

# print results of the smoothing
print("Classification results before:")

acc_before = metrics.accuracy_score(y_test['Event'], predictions)
print("Accuracy:" )
print(acc_before)

vis_clfs(y_test['Event'],predictions, mode="string")


print("Classification results after:")


acc_after = metrics.accuracy_score(number_targets, improved_predict)
print("Accuracy:" )
print(acc_after)

vis_clfs(number_targets,improved_predict, mode='int')

## Training the classifier with data from the other subjects

After the good results we gained for each subject separately we want to explore now how much we can improve the predictions using also the data of the other subjects. For this we split the whole dataset in training and test set. The results of the analysis below are captured in the following table.

As we can see we gain a much better prediction accuracy for the 1 second snippets (98+%) than for the 30 second snippets ((89+%). As argued also above this might be due to overfitting, since neighouring datapoints show the same characteristics.

In order to explore the limits of the classifiers we also tested additionally to the 67/33 train/test datasplit a 10/90 train/test datasplit. We can see that the performance of both classifiers declines in both cases (1 sec and 30 sec snippets) but declines less in the case of the 1 sec snippets. However it is remarkable how little the performance declines if we take into account how much we reduce the dataset.

<img src="table_within_scores.png" width=80%>

In [ ]:
features, targets = load_features(snippets = 1, drop = True)
X_train,X_test,y_train,y_test = train_test_split(features, targets, test_size=0.9, random_state=0)


In [ ]:
# Hyperparameter Search for one subject in order to approximate the best model
print("The best three models for the random forest are: \n")
do_random_search(X_train,y_train, classifier = 'rf')

print("The best three models for the AdaBoost are: \n")
do_random_search(X_train,y_train, classifier = 'ada')

Running the Classifiers on the test set: -> confusion matrix

In [ ]:
# insert parameters that worked best - see above
rf_clf = ensemble.RandomForestClassifier(max_depth = None, n_estimators = 41, criterion='entropy', class_weight='balanced', max_features=0.75, n_jobs = -1)
ada_clf = ensemble.AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=3),n_estimators=178, algorithm ='SAMME')


rf_clf.fit(X_train,y_train['Event'])
ada_clf.fit(X_train,y_train['Event'])


rf_pred_test = rf_clf.predict(X_test)
ada_pred_test = ada_clf.predict(X_test)


rf_acc_test = metrics.accuracy_score(y_test['Event'], rf_pred_test)

print("This is the Score for Random Forest on the test set: {}".format(rf_acc_test))

ada_acc_test = metrics.accuracy_score(y_test['Event'], ada_pred_test)
print("This is the Score for Ada-Boost on the test set: {} \n \n".format(ada_acc_test))

rf_cnf_matrix_test = confusion_matrix(y_test['Event'], rf_pred_test)
ada_cnf_matrix_test = confusion_matrix(y_test['Event'], ada_pred_test)

class_names, counts = np.unique(y_train['Event'], return_counts=True)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10))
plt.subplot(121)
plot_confusion_matrix(rf_cnf_matrix_test, classes=class_names,
                      title='Confusion matrix for the Random Forest for the test set')

# Plot normalized confusion matrix
plt.subplot(122)
plot_confusion_matrix(ada_cnf_matrix_test, classes=class_names, normalize=False,
                      title='Confusion matrix for the Random Forest for test set')

plt.show()

## Classification between subjects

### Create a separate test set to test our classifiers on
We split one subject and keep it as a separate test set.

In [ ]:
# get the feature and target table
features, targets = load_features(snippets = 30, drop = True)


#Test set subject a
X_test = features[features['40']==0]
y_test = targets[targets['subject_id']==0]

# Training set
X_train = features[features['40'] > 0]
y_train =  targets[targets['subject_id'] > 0]



### Crossvalidation
The general set up consists of a 4 fold crossvalidation with always one subject separated

In [ ]:

# only 5 subjects in the dataset
cv_labels = y_train['subject_id']
lkf = LabelKFold(cv_labels, n_folds=4)


### Hyperparameter Search

- results of hyperparameter search for 1s, 3s an d30 s

In [ ]:
# Hyperparameter Search for one subject in order to approximate the best model
print("The best three models for the random forest are: \n")
do_random_search(X_train,y_train, classifier = 'rf', cv = lkf)

print("The best three models for the AdaBoost are: \n")
do_random_search(X_train,y_train, classifier = 'ada', cv = lkf)

### Results for the Test set
- leave out channels
- 1s, 3s, 30s
Ada vs Forest -> how good

In [ ]:
# insert parameters that worked best - see above
rf_clf = ensemble.RandomForestClassifier(max_depth = None, n_estimators = 52, criterion='entropy', class_weight='balanced', max_features=0.1, n_jobs = -1)
ada_clf = ensemble.AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=1),n_estimators=156, algorithm ='SAMME')


rf_clf.fit(X_train,y_train['Event'])
ada_clf.fit(X_train,y_train['Event'])


rf_pred_test = rf_clf.predict(X_test)
ada_pred_test = ada_clf.predict(X_test)


rf_acc_test = metrics.accuracy_score(y_test['Event'], rf_pred_test)
print("This is the Score for Random Forest on the test set: {}".format(rf_acc_test))

ada_acc_test = metrics.accuracy_score(y_test['Event'], ada_pred_test)
print("This is the Score for Ada-Boost on the test set: {} \n \n".format(ada_acc_test))

rf_cnf_matrix_test = confusion_matrix(y_test['Event'], rf_pred_test)
ada_cnf_matrix_test = confusion_matrix(y_test['Event'], ada_pred_test)

class_names, counts = np.unique(y_train['Event'], return_counts=True)

# Plot non-normalized confusion matrix
plt.figure(figsize=(20,10))
plt.subplot(121)
plot_confusion_matrix(rf_cnf_matrix_test, classes=class_names,
                      title='Confusion matrix for the Random Forest for the test set')

# Plot normalized confusion matrix
plt.subplot(122)
plot_confusion_matrix(ada_cnf_matrix_test, classes=class_names, normalize=False,
                      title='Confusion matrix for the Random Forest for test set')

plt.show()

### Discussion for separate Subject prediction

- not enough data 
- generalizing is tricky: 
    - maybe intersubject differences: 
    - inter and intra: location of electrodes, bluetooth noise other noise etc
- with more data probably better predictions
- even take into account only a small portion of the new data should make it better

## Conclusion

• Wavelet transform allows to adapt and extract relevant information from both time and frequency domain. Because some sleep stages are mainly characterized by the time domain while others by the frequency domain, wavelet decomposition is most appropriate for extracting frequency bands from EEG sleep data.   • Random Forest and Adaboost preform both over xxx • Random Forest outperforms Adaboost algorithm by xxx • Hyperparameter search optimizes the results from both methods significantly. • Smoothing classification results within sleep stages can help to improve overall results